# HAND INDEX procedure

Here, add a description of this thing!

## 0. Initalize

- import required modules
- set paths to source files
- obtain propertis of input DEM

In [1]:
from os.path import join

class MyPath:
    """Creates path for various files"""
    def __init__(self, main_dir, file_name):
        self.main_dir = main_dir
        self.main_name = file_name
        self.path_wo_ext = join(main_dir, file_name)
    
    def tif(self, extension=None):
        if extension:
            new_path = self.path_wo_ext + extension + ".tif"
        else:
            new_path = self.path_wo_ext + ".tif"
        return new_path

In [2]:
import subprocess
import os
import rasterio
from osgeo import gdal
from pyproj.crs import CRS

In [9]:
# # Path to gdal command line programs
# path_gdal = "C:\\Program Files\\GDAL"

# Set output directory and name
case_dir = "full_slo_00"
# case_dir = "raw_sample"
work_dir = "C:\\Users\\ncoz\\ARRS_susa"
out_dir = os.path.join(work_dir, case_dir)
os.makedirs(out_dir, exist_ok=True)

# Main name for out files, e.g. dem_<out_nam>_ext.tif
out_nam = "si_mosaic_00"

# Input DEM
dem_original = "c:\\Users\\ncoz\\ARRS_susa\\dmv_10m.tif"

# Input SHP files
stream_in = "c:\\Users\\ncoz\\ARRS_susa\\DRSV_HIDRO5_LIN_PV\\DRSV_HIDRO5_LIN_PV.shp"
nodes_in = "c:\\Users\\ncoz\\ARRS_susa\\DRSV_HIDRO5_LIN_PV\\DRSV_HIDRO5_LIN_PV_vertices.shp"

# Create path objects
dem = MyPath(out_dir, "dem_" + out_nam)
stream = MyPath(out_dir, "streams_" + out_nam)
nodes = MyPath(out_dir, "nodes_" + out_nam)

# TauDEM MPI settings to enable parallel computations
mpi_settings = ["mpiexec", "-n", "8"]

# Get extent and epsg code of dem
with rasterio.open(dem_original) as ds:
    dem_bb = ds.bounds
    dem_res = ds.res
    dem_meta = ds.profile

# Assign CRS from pyproj
slo_crs = CRS.from_epsg(3794)

print("Check paths:")
if os.path.exists(dem_original):
    print(f"DEM    : {dem_original}")
if os.path.exists(stream_in):
    print(f"Nodes  : {stream_in}")
if os.path.exists(nodes_in):
    print(f"Streams: {nodes_in}")
print("\nCheck DEM propertis:")
print(dem_bb)
print(dem_res)
print("\nOutputs will be stored to:")
print(out_dir)

Check paths:
DEM    : c:\Users\ncoz\ARRS_susa\dmv_10m.tif
Nodes  : c:\Users\ncoz\ARRS_susa\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV.shp
Streams: c:\Users\ncoz\ARRS_susa\DRSV_HIDRO5_LIN_PV\DRSV_HIDRO5_LIN_PV_vertices.shp

Check DEM propertis:
BoundingBox(left=374000.0, bottom=31000.0, right=624010.0, top=195000.0)
(10.0, 10.0)

Outputs will be stored to:
C:\Users\ncoz\ARRS_susa\full_slo_00


## 1. Rasterize shape files

- streams file (line features)
- nodes file (point features)

In [10]:
# Rasterize streams to match DEM
subprocess.call(
    " ".join([
        "gdal_rasterize",
        "-burn", "1",
        "-co", "COMPRESS=LZW",
        "-init", "0",
        "-tap",
        "-ot", "Byte",
        "-te", " ".join([str(i) for i in list(dem_bb)]),
        "-tr", str(dem_res[0]) + " " + str(dem_res[1]),
        stream_in,
        stream.tif()
    ])
)

0

In [11]:
# Rasterize vertices to match DEM
subprocess.call(
    " ".join([
        "gdal_rasterize",
        "-burn", "1",
        "-co", "COMPRESS=LZW",
        "-init", "0",
        "-tap",
        "-ot", "Byte",
        "-te", " ".join([str(i) for i in list(dem_bb)]),
        "-tr", str(dem_res[0]) + " " + str(dem_res[1]),
        nodes_in,
        nodes.tif()
    ])
)

0

## 1. Preparation of the elevation model

### Burn streams into DEM (optional)

Basically we force breach any potential obsticals in the downhill path of the streams.

IN:
- original (input) DEM
- rasterized streams `strem.tif`

OUT:
- DEM with burned-in streams `dem_burned.tif`

In [12]:
# Burn streams into dem
src_a = rasterio.open(dem_original)
src_b = rasterio.open(stream.tif())

# Create output raster
output_file = dem.tif("_burn")
out_meta = src_a.profile.copy()

grid_a = src_a.read()
grid_b = src_b.read()

print(grid_a.shape)
print(grid_b.shape)

grid_out = (grid_a - (grid_b * 100))

out_meta.update({"crs":slo_crs})

with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(grid_out)

src_a.close()
src_b.close()

print("DONE!")

(1, 16400, 25001)
(1, 16400, 25001)
DONE!


### Fill pits
Filling pits prevents localized depressions and fill in the burned-in streams from previous step. If "burn" step was skipped, fill pits on the source DEM.

IN:
- DEM with burned in rasterized strems: `dem_burn.tif`

OUT:
- Pit-filled DEM with burned in streams: `dem_burn_fel.tif`

In [13]:
# Fill pits
subprocess.call(
    mpi_settings + [
        "pitremove", 
        "-z", dem.tif("_burn"),
        "-fel", dem.tif("_burn_fel")
    ]
)
with rasterio.open(dem.tif("_burn_fel"), "r+") as rio:
    rio.crs = slo_crs

### Determine flow directions
Gives two rasters. In the first one, each pixel has value 1-8, representing the direction of the flow in that pixel. The other raster provides slope angles for each pixel.

IN:
- Pit-filled DEM with burned in streams: `dem_burn_fel.tif`

OUT:
- Flow directions: `dem_burn_fel_p.tif`
- Slope: `dem_burn_fel_sd8.tif`

In [ ]:
# Determine flow dirs
subprocess.call(
    mpi_settings + [
        "d8flowdir",
        "-fel", dem.tif("_burn_fel"),
        "-p", dem.tif("_burn_fel_p"),
        "-sd8", dem.tif("_burn_fel_sd8")
    ]
)
with rasterio.open(dem.tif("_burn_fel_p"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_burn_fel_sd8"), "r+") as rio:
    rio.crs = slo_crs

### Mask streams

In [10]:
# Mask flow dirs by streams
src_a = rasterio.open(dem.tif("_burn_fel_p"))
src_b = rasterio.open(stream.tif())

# Create output raster
output_file = dem.tif("_burn_fel_p_masked")
out_meta = src_a.profile.copy()

grid_a = src_a.read()
grid_b = src_b.read()

print(grid_a.shape)
print(grid_b.shape)

grid_out = (grid_a * grid_b)

out_meta.update({"crs":slo_crs,
                 "dtype":'int16'})

with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(grid_out)

src_a.close()
src_b.close()

print("DONE!")

(1, 1700, 1700)
(1, 1700, 1700)
DONE!


### Conditioned dem
This is final product of this step.

In [11]:
# Condition DEM
subprocess.call(
    mpi_settings + [
        "flowdircond",
        "-z", dem_original,
        "-p", dem.tif("_burn_fel_p_masked"),
        "-zfdc", dem.tif("_cond")
    ]
)
with rasterio.open(dem.tif("_cond"), "r+") as rio:
    rio.crs = slo_crs

## 2. Pre-processing

Using the conditioned elevation model we adopt the traditional steps of filling any remaining pits, calculating flow directions and delineating a stream network using the start/end points as weights. This stream network is close to the observed stream network I downloaded, with the difference that this is consistent with flow directions obtained from the digital elevation model.

### Remove pits on conditioned DEM

Use the first cell if pre-processing of DEM was skipped.

The Second cell is default for a preprocessed DEM.

In [ ]:
# USE THIS CELL IF THE PRE-PROCESSING OF DEM HAS BEEN SKIPPED
subprocess.call(
    mpi_settings + [
        "pitremove", 
        "-z", "-z", dem.tif("_cond"),,
        "-fel", dem.tif("_cond_fel")
    ]
)
with rasterio.open(dem.tif("_cond_fel"), "r+") as rio:
    rio.crs = slo_crs

In [7]:
# Remove pits on conditioned DEM
subprocess.call(
    mpi_settings + [
        "pitremove", 
        "-z", "-z", dem.tif("_cond"),,
        "-fel", dem.tif("_cond_fel")
    ]
)
with rasterio.open(dem.tif("_cond_fel"), "r+") as rio:
    rio.crs = slo_crs

### Determine flow directions on conditioned DEM
Standar procedure.

In [8]:
# Determine flow dirs on conditioned dem
subprocess.call(
    mpi_settings + [
        "d8flowdir",
        "-fel", dem.tif("_cond_fel"),
        "-p", dem.tif("_cond_fel_p"),
        "-sd8", dem.tif("_cond_fel_sd8")
    ]
)
with rasterio.open(dem.tif("_cond_fel_p"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_cond_fel_sd8"), "r+") as rio:
    rio.crs = slo_crs

### Flow accumulation

In [9]:
# Flow accumulation on conditioned dem
subprocess.call(
    mpi_settings + [
        "aread8", 
        "-p", dem.tif("_cond_fel_p"),
        "-ad8", dem.tif("_cond_fel_p_ad8")])
with rasterio.open(dem.tif("_cond_fel_p_ad8"), "r+") as rio:
    rio.crs = slo_crs

### Flow accumulation weighted by stream nodes

Here is the only place I am using strem nodes extrated from stream vector file.

In [10]:
# Flow accumulation weighted by stream start and end points
subprocess.call(
    mpi_settings + [
        "aread8", 
        "-p", dem.tif("_cond_fel_p"),
        "-ad8", dem.tif("_cond_fel_p_ssa"),
        "-wg", nodes.tif()
    ]
)
with rasterio.open(dem.tif("_cond_fel_p_ssa"), "r+") as rio:
    rio.crs = slo_crs

### Delineate streams by threshold
In literature threshold 50 is suggested?!?!

In [11]:
# Delineate streams by threshold
subprocess.call(
    mpi_settings + [
        "threshold", 
        "-ssa", dem.tif("_cond_fel_p_ssa"),
        "-src", dem.tif("_cond_fel_p_ssa_src"),
        "-thresh", "1"
    ]
)
with rasterio.open(dem.tif("_cond_fel_p_ssa_src"), "r+") as rio:
    rio.crs = slo_crs

## Calculate HAND

Finally, we calculate D-Inf flow directions and HAND. The result is a grid where each cell denotes the vertical height above the nearest stream, with “nearest” being the nearest stream cell in along the D-Inf flowpath. In the final step, we use some raster math to assign zero to the coastal area for which HAND has not been calculated.

In [12]:
# Calculate infinity flow directions
subprocess.call(
    mpi_settings + [
        "dinfflowdir",
        "-fel", dem.tif("_cond_fel"),
        "-slp", dem.tif("_cond_fel_slp"),
        "-ang", dem.tif("_cond_fel_ang")
    ]
)
with rasterio.open(dem.tif("_cond_fel_slp"), "r+") as rio:
    rio.crs = slo_crs
with rasterio.open(dem.tif("_cond_fel_ang"), "r+") as rio:
    rio.crs = slo_crs

In [13]:
# Calculate HAND raster
subprocess.call(
    mpi_settings + [
        "dinfdistdown",
        "-fel", dem.tif("_cond_fel"),
        "-slp", dem.tif("_cond_fel_slp"),
        "-ang", dem.tif("_cond_fel_ang"),
        "-src", dem.tif("_cond_fel_p_ssa_src"),
        "-dd", dem.tif("_cond_HAND"),
        "-m",  "v",  "ave"
    ]
)

0

### Some post-processig to make it look nicer
Set nodata to 0, lso we dont get wird gaps on the edges.

In [14]:
# Fill HAND raster with zero on land surfaces where HAND is nodata
with rasterio.open(dem.tif("_cond_HAND")) as grid:
    handgrid = grid.read()
    handmeta = grid.profile

handgrid[handgrid < 0] = 0

# This one is used to set all nodata from original dem to -1
with rasterio.open(dem_original) as grid:
    demgrid = grid.read()
    demmeta = grid.profile

handgrid[demgrid == -9999] = -1

handmeta["nodata"] = -1
handmeta.update({"crs":slo_crs})

with rasterio.open(dem.tif("_cond_HAND_zeroes"), "w", **handmeta) as dst:
    dst.write(handgrid)

print("DONE!")

DONE!
